In [1]:
#import libraries
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/Data"


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

!pip install antspyx
import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json

In [2]:
#import data
sample_path = "/home/jovyan/Data/dataset" 
import os
print(os.path.exists(sample_path))
dataset_demo= "ds1610" 
bids_root = op.join(os.path.abspath(""),sample_path, dataset_demo) 
derivatives = op.join(os.path.abspath(""),bids_root, "derivatives/preprocessed_data/subject101410") 
mkdir_no_exist(derivatives)
LR_Motor = bids_root + '/subject101410/fMRI/tfMRI_MOTOR_LR/tfMRI_MOTOR_LR.nii' 
RL_Motor = bids_root + '/subject101410/fMRI/tfMRI_MOTOR_RL/tfMRI_MOTOR_RL.nii' 
print_dir_tree(bids_root, max_depth=5) 

True
|ds1610/
|--- .ipynb_checkpoints/
|--- derivatives/
|------ preprocessed_data/
|--------- subject101410/
|--- subject101410/
|------ dataset_description.md
|------ task-motor_bold.json
|------ .ipynb_checkpoints/
|------ fMRI/
|--------- .ipynb_checkpoints/
|--------- tfMRI_MOTOR_LR/
|------------ events_LR.csv
|------------ tfMRI_MOTOR_LR.nii
|--------- tfMRI_MOTOR_RL/
|------------ tfMRI_MOTOR_RL.nii
|------------ .ipynb_checkpoints/


In [3]:
#strcutural preporcessing
#fsleyesDisplay = FSLeyesServer() 
#fsleyesDisplay.show() 
#fsleyesDisplay.resetOverlays() 
#fsleyesDisplay.load(LR_Motor)

Gtk-Message: 10:16:29.975: Failed to load module "canberra-gtk-module"


In [10]:
#functional preprocessing
import nibabel as nib
import numpy as np
import os

def load_img_data(path):
    img = nib.load(path)
    data = img.get_fdata(dtype=np.float32)  # shape = (X, Y, Z, T)
    return img, data

def brain_mask_from_nonzero(data, threshold=1e-6):
    """
    Create a brain mask based on voxels with mean absolute intensity > threshold.
    The threshold is tiny, so it excludes background zeros but keeps all real voxels.
    """
    mean_signal = np.mean(np.abs(data), axis=3)
    mask = mean_signal > threshold
    return mask

def global_variance_over_mask(data, mask):
    """Compute global variance across all brain voxels and timepoints."""
    brain_ts = data[mask]  # shape = (n_voxels, T)
    return np.var(brain_ts)

# ---- File paths ----
run1 = LR_Motor
run2 = RL_Motor

# ---- Load both runs ----
img1, d1 = load_img_data(run1)
img2, d2 = load_img_data(run2)

print(img1.shape)
print(img2.shape)

print(global_variance_over_mask(d1, mask1))
print(global_variance_over_mask(d2, mask2))

(91, 109, 91, 284)
(91, 109, 91, 284)
11232342.0
10685729.0


In [5]:
# ---- Make masks (using small threshold) ----
mask1 = brain_mask_from_nonzero(d1, threshold=1e-6)
mask2 = brain_mask_from_nonzero(d2, threshold=1e-6)

# ---- Compute global variance for each run ----
var1 = global_variance_over_mask(d1, mask1)
var2 = global_variance_over_mask(d2, mask2)

# ---- Rescale so variance = 1 ----
d1_scaled = d1 / np.sqrt(var1)
d2_scaled = d2 / np.sqrt(var2)

print(global_variance_over_mask(d1_scaled, mask1))
print(global_variance_over_mask(d2_scaled, mask2))


0.99999756
1.0000005


In [13]:
# ---- Concatenate along time (4th dimension) ----
concat = np.concatenate((d1_scaled, d2_scaled), axis=3)

print(concat.shape)
print(global_variance_over_mask(concat, mask1))

(91, 109, 91, 568)
1.0014294



(ipykernel_launcher.py:1140): GLib-GIO-CRITICAL **: 10:29:59.017: GFileInfo created without standard::is-hidden

(ipykernel_launcher.py:1140): GLib-GIO-CRITICAL **: 10:29:59.018: file ../gio/gfileinfo.c: line 1633 (g_file_info_get_is_hidden): should not be reached

(ipykernel_launcher.py:1140): GLib-GIO-CRITICAL **: 10:29:59.018: GFileInfo created without standard::is-backup

(ipykernel_launcher.py:1140): GLib-GIO-CRITICAL **: 10:29:59.018: file ../gio/gfileinfo.c: line 1655 (g_file_info_get_is_backup): should not be reached

(ipykernel_launcher.py:1140): GLib-GIO-CRITICAL **: 10:31:59.034: GFileInfo created without standard::is-hidden

(ipykernel_launcher.py:1140): GLib-GIO-CRITICAL **: 10:31:59.034: file ../gio/gfileinfo.c: line 1633 (g_file_info_get_is_hidden): should not be reached

(ipykernel_launcher.py:1140): GLib-GIO-CRITICAL **: 10:31:59.034: GFileInfo created without standard::is-backup

(ipykernel_launcher.py:1140): GLib-GIO-CRITICAL **: 10:31:59.034: file ../gio/gfileinfo.

In [7]:
# ---- Save output ----
out_path = derivatives+"/fMRI_motor_concat_var1.nii"

out_img = nib.Nifti1Image(concat, img1.affine, img1.header)
out_img.to_filename(out_path)

print(f"✅ Saved concatenated, variance-normalized fMRI:\n{out_path}")

✅ Saved concatenated, variance-normalized fMRI:
/home/jovyan/Data/dataset/ds1610/derivatives/preprocessed_data/subject101410/fMRI_motor_concat_var1.nii


In [9]:
fsleyesDisplay = FSLeyesServer() 
fsleyesDisplay.show() 
fsleyesDisplay.resetOverlays() 
fsleyesDisplay.load(LR_Motor)


(ipykernel_launcher.py:1140): Gdk-WARNING **: 10:24:36.973: gdkdrawable-x11.c:952 drawable is not a pixmap or window


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>


(ipykernel_launcher.py:1140): GLib-GIO-CRITICAL **: 10:25:58.960: GFileInfo created without standard::is-hidden

(ipykernel_launcher.py:1140): GLib-GIO-CRITICAL **: 10:25:58.960: file ../gio/gfileinfo.c: line 1633 (g_file_info_get_is_hidden): should not be reached

(ipykernel_launcher.py:1140): GLib-GIO-CRITICAL **: 10:25:58.960: GFileInfo created without standard::is-backup

(ipykernel_launcher.py:1140): GLib-GIO-CRITICAL **: 10:25:58.960: file ../gio/gfileinfo.c: line 1655 (g_file_info_get_is_backup): should not be reached


In [ ]:
#coregistration

In [ ]:
#Gaussian Smoothing